### 카카오 매장 블로그 후기 검색

In [5]:
# 필요한 모듈 참조
import requests
import json
from pandas import DataFrame
from pandas import read_excel 
import pymysql
from sqlalchemy import create_engine
from pandas import DataFrame
from pandas import read_sql, read_sql_table
# 데이터베이스 접속 패키지 설치
pymysql.install_as_MySQLdb() 
import MySQLdb

In [6]:
# 접속 문자열 생성
# mysql+mysqldb://계정이름:비밀번호@:포트번호/데이터베이스이름?charset=인코딩
conStr = 'mysql+mysqldb://root:1234@:3306/semiproject?charset=utf8' # utf8db 디비명 
# 데이터베이스 접속하기
engine = create_engine(conStr)
conn = engine.connect() # conn ; 접속객체

In [7]:
# 접속할 URL
urlFmt = "https://dapi.kakao.com/v2/search/blog?query={query}#&page={page}&size={size}"


In [8]:
# 검색할 지역
location = '서울특별시 서초구 방배동 836-10 삼경빌딩'
loc = location.split(" ")
loc


['서울특별시', '서초구', '방배동', '836-10', '삼경빌딩']

In [9]:
# 상위 100개 데이터 가져오기 
df = read_sql('SELECT * FROM yogiyo_rank',index_col='매장명',con=conn)
rank100 = df.head(100)
rank100  = rank100.index
rank100

Index(['기대만족-본점', '벚꽃스시-상도점', '벚꽃스시-상도점', '반올림피자-사당방배점', '반올림피자-사당방배점',
       '반올림피자-사당방배점', '피자나라치킨공주-서울봉천점', '피자나라치킨공주-서울봉천점', '피자나라치킨공주-서울봉천점',
       '피자나라치킨공주-서울봉천점', '피자나라치킨공주-서울봉천점', '청년피자-사당방배점', '청년피자-사당방배점',
       '청년피자-사당방배점', '버거킹-사당역점', '버거킹-사당역점', 'BHC-사당점', 'BHC-사당점', 'BHC-사당점',
       '마라하오-서초점', '마라하오-서초점', '마라하오-서초점', '마라하오-서초점', '황궁쟁반짜장-서경', '타이반쩜-교대점',
       '타이반쩜-교대점', '타이반쩜-교대점', 'BHC-이수점', 'BHC-이수점', 'BHC-이수점', 'BHC-이수점',
       'BHC-이수점', '회통령', '회통령', '교촌치킨-사당3호점', '교촌치킨-사당3호점', '교촌치킨-사당3호점',
       '달떡볶이-강남점', '달떡볶이-강남점', '달떡볶이-강남점', '달떡볶이-강남점', '달떡볶이-강남점', '달떡볶이-강남점',
       '꾸브라꼬숯불두마리치킨-강남점', '꾸브라꼬숯불두마리치킨-강남점', '꾸브라꼬숯불두마리치킨-강남점',
       '꾸브라꼬숯불두마리치킨-강남점', '장모님이떴다-본점', '장모님이떴다-본점', '장모님이떴다-본점', '장모님이떴다-본점',
       '스쿨푸드-방배점', '스쿨푸드-방배점', '스쿨푸드-방배점', '스쿨푸드-방배점', '스쿨푸드-방배점', '롯데리아-이수역점',
       '롯데리아-이수역점', '롯데리아-이수역점', '롯데리아-이수역점', '롯데리아-이수역점', '문숙이미나리식당-강남직영점',
       '문숙이미나리식당-강남직영점', '문숙이미나리식당-강남직영점', '조이플', '취홍-서울교대점', '스쿨푸드-서초점',
       '스쿨푸드-서초점', '스쿨

In [59]:
# 검색 매장

store_list = list(set(rank100))
print(len(store_list),'개의 매장을 검색합니다',store_list)

# 최종 저장할 리스트 생성 
beast_store_list = []


32 개의 매장을 검색합니다 ['반올림피자-사당방배점', '마라하오-서초점', '청년피자-사당방배점', 'BHC-방배본동점', '베트남쌀국수미스사이공-이수점', '스쿨푸드-방배점', '두마리찜닭두찜-서초방배점', '롯데리아-이수역점', 'BHC-사당점', '회통령', '스쿨푸드-서초점', '냠냠족발&보쌈-강남역삼점', '청년치킨-사당점', '벚꽃스시-상도점', '동원-방배점', '교촌치킨-방배1호점', '기대만족-본점', '황궁쟁반짜장-서경', '꾸브라꼬숯불두마리치킨-강남점', '문숙이미나리식당-강남직영점', '피자나라치킨공주-서울봉천점', '조이플', '지코바치킨-서초1호점', '교촌치킨-사당3호점', '장모님이떴다-본점', 'BHC-이수점', '미스터보쌈족발-낙성대점', '타이반쩜-교대점', '달떡볶이-강남점', '취홍-서울교대점', '모두뼈다귀감자탕-상도점', '버거킹-사당역점']


In [60]:
for i in range(len(store_list)) : # 매장 숫자만큼 반복 28번 

    # 요청변수
    query = f"{loc[1]} {loc[2]} {store_list[i]}" # 쿼리에 매장 정보를 날려 
    page = 1
    size = 50
    key = "87fa270de879f3dd014b8779623884db" 

    # 접속 객체 생성
    session = requests.Session()

    # 접속객체에 부가정보(header) 삽입하기
    session.headers.update({
        "Authorization": "KakaoAK %s" % key    # 인증키
    })

    # 다음 페이지가 존재하는지 여부 --> False인 동안은 while문 반복
    isEnd = False

    # 반복 수행 도중 추출되는 검색결과 리스트를 결합할 빈 리스트 준비
    mylist = []

    # 1페이지부터 시작
    page = 1

    while (not isEnd) :            
        # 접속할 URL
        url = urlFmt.format(query=query, page=page, size=size)
        url

        # 생성한 접속객체를 활용하여 API에 접속
        r = session.get(url)

        # 접속에 실패한 경우
        if r.status_code != 200:
            break
            # # 에러코드와 에러메시지 출력
            # msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            # # 에러를 강제로 생성시킴
            # raise Exception(msg)
                

        # 인코딩 형식 지정
        r.encoding = "utf-8"
        # 가져온 문자열을 딕셔너리로 변환
        data = json.loads(r.text)        

        # 검색 결과를 갖고 있는 딕셔너리를 포함하는 리스트 추출
        searchList = data['documents']
        
        for j in range(len(searchList)) :
            searchList[j]['매장명'] = store_list[i]
            searchList[j]['contents'] = (searchList[j]['contents'].replace('</b>','')).replace('<b>','')
            searchList[j]['title'] = (searchList[j]['title'].replace('</b>','')).replace('<b>','')

        # 추출한 리스트를 미리 준비한 빈 리스트에 추가
        mylist.extend(searchList)

        # 증감식에 해당하는 isEnd변수 추출
        isEnd = bool(data['meta']['is_end'])
        page += 1
        print("isEnd=", isEnd, ", 다음페이지=", page)

        if page == 5 :
            isEnd = True
    
    print("최종 검색 건수: %d" % len(mylist))

    # 검색 결과를 가족 있는 딕셔너리를 포함하는 리스트를 추출한다. 
    searchList = data['documents']

    # 추출한 리스트를 미리 준비한 빈 리스트에 추가
    beast_store_list.extend(searchList) 
   

isEnd= False , 다음페이지= 2
isEnd= False , 다음페이지= 3
isEnd= False , 다음페이지= 4
isEnd= False , 다음페이지= 5
최종 검색 건수: 40
isEnd= True , 다음페이지= 2
최종 검색 건수: 3
isEnd= False , 다음페이지= 2
isEnd= False , 다음페이지= 3
isEnd= False , 다음페이지= 4
isEnd= False , 다음페이지= 5
최종 검색 건수: 40
isEnd= True , 다음페이지= 2
최종 검색 건수: 6
isEnd= False , 다음페이지= 2
isEnd= False , 다음페이지= 3
isEnd= False , 다음페이지= 4
isEnd= False , 다음페이지= 5
최종 검색 건수: 40
isEnd= False , 다음페이지= 2
isEnd= False , 다음페이지= 3
isEnd= False , 다음페이지= 4
isEnd= False , 다음페이지= 5
최종 검색 건수: 40
isEnd= False , 다음페이지= 2
isEnd= False , 다음페이지= 3
isEnd= False , 다음페이지= 4
isEnd= False , 다음페이지= 5
최종 검색 건수: 40
isEnd= True , 다음페이지= 2
최종 검색 건수: 6
isEnd= False , 다음페이지= 2
isEnd= False , 다음페이지= 3
isEnd= False , 다음페이지= 4
isEnd= False , 다음페이지= 5
최종 검색 건수: 40
isEnd= True , 다음페이지= 2
최종 검색 건수: 0
isEnd= False , 다음페이지= 2
isEnd= False , 다음페이지= 3
isEnd= False , 다음페이지= 4
isEnd= False , 다음페이지= 5
최종 검색 건수: 40
isEnd= False , 다음페이지= 2
isEnd= False , 다음페이지= 3
isEnd= False , 다음페이지= 4
isEnd= False , 다음페이지= 5
최

In [64]:
import pymysql
from sqlalchemy import create_engine
from pandas import DataFrame
from pandas import read_sql, read_sql_table
# 데이터베이스 접속 패키지 설치
pymysql.install_as_MySQLdb() #pip이 없음 
import MySQLdb

In [75]:
# 접속 문자열 생성
# mysql+mysqldb://계정이름:비밀번호@:포트번호/데이터베이스이름?charset=인코딩
conStr = 'mysql+mysqldb://root:1234@:3306/semiproject?charset=utf8' # utf8db 디비명 
# 데이터베이스 접속하기
engine = create_engine(conStr)
conn = engine.connect() # conn ; 접속객체



In [78]:
# 결과 엑셀 저장 및 출력
df = DataFrame(beast_store_list) 
df.drop(['thumbnail'],axis=1,inplace=True)
df.drop(['contents'],axis=1,inplace=True)
df.drop(['title'],axis=1,inplace=True)
df.set_index('매장명')
df.to_excel("카카오 블로그 검색 결과(%s).xlsx" % loc[2],sheet_name='검색',index=False)



In [82]:
# 데이터 내보내기
df.to_sql(name='daum',con=conn,if_exists='append',index=False) # new_table 테이블명 
conn.commit()

In [80]:
df = read_sql('SELECT * FROM daum',index_col='매장명',con=conn)
df

,blogname,contents,datetime,title,url
매장명,,,,,
반올림피자-사당방배점,해피수마일v,None,2023-06-20T08:39:00.000+09:00,None,https://blog.naver.com/haesoo7666/223133562444
반올림피자-사당방배점,방방곡곡 NK가 간다,None,2023-05-18T11:00:00.000+09:00,None,https://blog.naver.com/weperfect/223101916446
반올림피자-사당방배점,밤끼의 홍익먹방 실현공간,None,2023-06-17T11:53:00.000+09:00,None,https://blog.naver.com/bamkki_foodie/223131286001
반올림피자-사당방배점,데미안의 리뷰찾기,None,2023-04-11T13:27:00.000+09:00,None,https://blog.naver.com/vudod1/223070302832
반올림피자-사당방배점,하 루 하 루 기 록 하 기,None,2023-03-29T11:37:00.000+09:00,None,https://blog.naver.com/dms614614/223058674176
...,...,...,...,...,...
버거킹-사당역점,"한걸음, 한걸음",None,2020-12-04T11:31:00.000+09:00,None,https://blog.naver.com/chocochip121/222162528760
버거킹-사당역점,MINGKEY's blog,None,2020-11-10T00:13:00.000+09:00,None,https://blog.naver.com/qkrpsla/222140087260
버거킹-사당역점,보통의 일상,None,2019-05-31T17:43:00.000+09:00,None,https://blog.naver.com/jellran/221551350261
